# Calculating GHG at the Ethnologue Polygon Level

In [8]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import xarray as xr
import rioxarray
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
import matplotlib.patches as mpatches
from matplotlib.font_manager import FontProperties

import mapclassify

from rapidfuzz import process, fuzz

from shapely.geometry import Point

import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterstats import zonal_stats

In [9]:
# Set base project path
base_path = Path("C:/Users/juami/Dropbox/RAships/2-Folklore-Nathan-Project/EA-Maps-Nathan-project/Measures_work")

# Set file paths
poscol_path = base_path / "data" / "raw" / "ethnologue" / "ancestral_characteristics_database_language_level" / "Ethnologue_16_shapefile" / "langa_no_overlap_biggest_clean.shp"

data_path = base_path / "data" / "interim"
maps_path = base_path / "maps" / "raw"
ghg_path = maps_path / "CO2" / "EDGAR_2025_GHG_CO2_2000_TOTALS_flx.nc"

In [10]:
import xarray as xr

ghg_path = maps_path / "CO2" / "EDGAR_2025_GHG_CO2_2000_TOTALS_flx.nc"
ds = xr.open_dataset(ghg_path)
print(ds)


<xarray.Dataset> Size: 26MB
Dimensions:  (lat: 1800, lon: 3600)
Coordinates:
  * lat      (lat) float64 14kB -89.95 -89.85 -89.75 ... 89.75 89.85 89.95
  * lon      (lon) float64 29kB -179.9 -179.8 -179.8 ... 179.8 179.8 179.9
Data variables:
    fluxes   (lat, lon) float32 26MB ...
Attributes:
    description:       TOTALS
    institution:       European Commission, Joint Research Centre
    source:            https://edgar.jrc.ec.europa.eu/dataset_ghg2025
    how_to_cite:       https://edgar.jrc.ec.europa.eu/dataset_ghg2025#howtocite
    copyright_notice:  https://edgar.jrc.ec.europa.eu/dataset_ghg2025#conditions
    contacts:          https://edgar.jrc.ec.europa.eu/dataset_ghg2025#info JR...
    units:             kg m-2 s-1


In [11]:
print("Vars:", list(ds.data_vars))

Vars: ['fluxes']


In [12]:
# --- Open and inspect ---
ds = xr.open_dataset(str(ghg_path), engine="netcdf4", decode_times=False)
da = ds["fluxes"]  # likely units: kg m-2 s-1 (check below)
print("attrs:", da.attrs)

# Ensure spatial dims are named and CRS is WGS84
da = da.rename({"lon":"lon", "lat":"lat"})  # no-op if already correct
da = da.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
da = da.rio.write_crs("EPSG:4326")

# --- Build per-pixel area on the sphere (m^2) from lat/lon edges ---
# Derive grid spacing in degrees
dlat = float(np.abs(np.diff(da["lat"].values).mean()))
dlon = float(np.abs(np.diff(da["lon"].values).mean()))

# Construct latitude edges (center +/- dlat/2), clipped to [-90, 90]
lat_c = da["lat"].values
lon_c = da["lon"].values
lat_edges = np.empty(lat_c.size + 1)
lat_edges[1:-1] = 0.5 * (lat_c[:-1] + lat_c[1:])
lat_edges[0] = lat_c[0] - dlat/2
lat_edges[-1] = lat_c[-1] + dlat/2
lat_edges = np.clip(lat_edges, -90.0, 90.0)

# Radians
deg2rad = np.pi / 180.0
lat_edges_rad = lat_edges * deg2rad
dlon_rad = dlon * deg2rad

# Earth radius (WGS84 mean)
R = 6371008.8

# Pixel area formula: A = R^2 * dlon * (sin(lat2) - sin(lat1))
# Make a 2D area array (lat x lon)
sin_lat2 = np.sin(lat_edges_rad[1:])[:, None]  # (nlat, 1)
sin_lat1 = np.sin(lat_edges_rad[:-1])[:, None] # (nlat, 1)
area_lat_strip = (R**2) * dlon_rad * (sin_lat2 - sin_lat1)  # (nlat, 1)
area = np.repeat(area_lat_strip, len(lon_c), axis=1)        # (nlat, nlon)

area_da = xr.DataArray(
    area,
    coords={"lat": da["lat"], "lon": da["lon"]},
    dims=("lat", "lon"),
    name="cell_area_m2"
)

# --- Convert flux (kg m-2 s-1) to tonnes per year per cell ---
# If your attrs say something different, adjust this step accordingly.
seconds_per_year = 365 * 24 * 3600  # 31,536,000; use 366 for leap years if needed
emi_kg_per_year = da * area_da * seconds_per_year
emi_tonnes_per_year = (emi_kg_per_year / 1000.0).rio.write_crs("EPSG:4326")
emi_tonnes_per_year.name = "CO2_tpy"



attrs: {'global_total': '25.69Gt', 'substance': 'CO2', 'year': '2000', 'release': 'EDGAR_2025_GHG', 'long_name': 'TOTALS', 'description': 'TOTALS', 'ChunkSizes': array([   1, 1800, 3600], dtype=int64), 'units': 'kg m-2 s-1'}


In [ ]:
# --- Write a GeoTIFF so we can use zonal_stats easily ---
# rioxarray knows the transform from coords once dims & CRS are set.
tif_path = Path(maps_path) / "CO2" / "EDGAR_2000_CO2_tpy_WGS84.tif"
emi_tonnes_per_year.rio.to_raster(tif_path)

# --- Read polygons (project to WGS84 to match the raster) ---
ethno = gpd.read_file(poscol_path).to_crs("EPSG:4326")

# --- Zonal sum (total tonnes/year per polygon) ---
zs = zonal_stats(
    ethno, str(tif_path),
    stats=["sum", "mean", "count"],
    geojson_out=False,
    nodata=None
)

# Attach results back to your GeoDataFrame
ethno["CO2_tpy_sum"] = [z["sum"] for z in zs]
ethno["CO2_tpy_mean"] = [z["mean"] for z in zs]
ethno["n_pixels"] = [z["count"] for z in zs]

# Save if you want
out_path = Path(maps_path) / "CO2" / "ethnologue_CO2_2000_tpy.geojson"
ethno.to_file(out_path, driver="GeoJSON")
print("Done ->", out_path)
